In [2]:
import cv2
import numpy as np

In [3]:
def pyrDown_custom(imagen, sigma):
    filtro = crearFiltroGaussiano(sigma)
    imagen = cv2.filter2D(imagen, -1, filtro)
    imagen = cv2.resize(imagen, (0,0), fx=0.5, fy=0.5)
    return imagen

def crearFiltroGaussiano(sigma):
    filter_size = 5
    gaussian_filter = np.zeros((filter_size, filter_size), np.float32)
    m = filter_size//2
    n = filter_size//2

    for x in range(-m, m+1):
        for y in range(-n, n+1):
            x1 = 2*np.pi*(sigma**2)
            x2 = np.exp(-(x**2 + y**2)/(2* sigma**2))
            gaussian_filter[x+m, y+n] = (1/x1)*x2
    return gaussian_filter

def calcularGauss(imagen, niveles, sigma, fourier = False)->list:
    copia = imagen.copy()
    piramide_gaussiana = [copia]
    for i in range(niveles):
        copia = pyrDown_custom(copia, sigma)
        piramide_gaussiana.append(copia)
    return piramide_gaussiana

def calcularLaplaciana(piramide_gaussiana):
    piramide_laplaciana = []
    for i in range(len(piramide_gaussiana)-1,0, -1):
        g_i = cv2.pyrUp(piramide_gaussiana[i]) #gi es la imagen incrementada en tamaño y la que deberia ser modificada si no hace match en tamaño
        g_i = ajustar_tamano(piramide_gaussiana[i-1], g_i)
        l_i = cv2.subtract(piramide_gaussiana[i-1], g_i)
        l_i[l_i < 0] = 0
        piramide_laplaciana.append(l_i)
    return piramide_laplaciana

def ajustar_tamano(img_norm, img_x2):
    canales = img_norm.shape[2]
    if img_x2.shape[0] < img_norm.shape[0]:
        empty_column = np.zeros((1, img_x2.shape[1], canales), dtype=np.uint8)
        img_x2 = np.vstack((img_x2, empty_column))
    elif img_x2.shape[0] > img_norm.shape[0]:
        img_x2 = img_x2[:-1, :, :]
    
    if img_x2.shape[1] < img_norm.shape[1]:
        empty_column = np.zeros((img_x2.shape[0], 1, canales), dtype=np.uint8)
        img_x2 = np.hstack((img_x2, empty_column))
    elif img_x2.shape[1] > img_norm.shape[1]:
        img_x2 = img_x2[:, :-1, :]
    return img_x2

def reconstruir(laplacianas, gaussianas, alpha, beta):
    img_r = gaussianas[-1]
    for i in range(len(gaussianas)):
        #print(i)
        try:
            laplaciana_i = laplacianas[-i-1]
        except:
            break
        img_r = cv2.pyrUp(img_r)
        img_r = ajustar_tamano(laplaciana_i, img_r)
        img_r = cv2.add(alpha*img_r, beta*laplaciana_i)
        cv2.imwrite("./reconstruida/img_reconstruida" + str(i) + ".jpg", img_r)

def guardar_imgs(laplacianas, gaussianas, nombre):
    for i in range(len(laplacianas)):
        cv2.imwrite("./piramides_"+ str(nombre)+"/laplaciana" + str(i) + ".jpg", laplacianas[i])
    for i in range(len(gaussianas)):
        cv2.imwrite("./piramides_"+ str(nombre) +"/gaussiana" + str(i) + ".jpg", gaussianas[i])

In [4]:
name_a = "taza4"
name_b = "taza3"
A = cv2.imread('{}.jpeg'.format(name_a))
B = cv2.imread('{}.jpeg'.format(name_b))


sigma = 0.6
ga = calcularGauss(A, 5, sigma, fourier = False)
gb = calcularGauss(B, 5, sigma, fourier = False)

la = calcularLaplaciana(ga)
lb = calcularLaplaciana(gb)

In [5]:
alpha = 0.85
beta = 1 - alpha
reconstruir(la[::-1], gb, alpha, beta)

In [6]:
guardar_imgs(la, ga, name_a)
guardar_imgs(lb, gb, name_b)

# Fourirer

In [ ]:
from numpy.fft import fft2, ifft2, fftshift

In [ ]:
def imagen_hibrida_fourier(image1, image2, cutoff_frequency, alpha):
    # Calcula la Transformada de Fourier de ambas imágenes
    fft_image1 = fft2(image1) 
    fft_image2 = fft2(image2)

    # Aplica un filtro en el dominio de Fourier a ambas imágenes
    height, width = image1.shape[:2] # Nos quedamos solo con las dimensiones de la imagen y NO con los canales
    
    h, w = height // 2, width // 2 # Coordenada del centro de la imagen

    # Eliminamos las frecuancias altas de la imagen 1. Utilizamos el hecho de que las frecuencias bajas
    # se encuentran cerca del centro de la imagen (en el dominio de las frecuencias)
    fft_image1[h-cutoff_frequency:h+cutoff_frequency, w-cutoff_frequency:w+cutoff_frequency] = 0

    # Eliminamos las frecuencias bajas de la imagen 2. Utilizamos el hecho de que las frecuencias bajas
    # se encuentran cerca del centro de la imagen (en el dominio de las frecuencias)
    fft_image2[:h-cutoff_frequency, :w-cutoff_frequency] = 0
    fft_image2[:h-cutoff_frequency, w+cutoff_frequency:] = 0
    fft_image2[h+cutoff_frequency:, :w-cutoff_frequency] = 0
    fft_image2[h+cutoff_frequency:, w+cutoff_frequency:] = 0

    # Calcula la Transformada Inversa de Fourier de ambas imágenes filtradas
    filtered_image1 = np.abs(ifft2(fft_image1)) # np.abs para eliminar la parte imaginaria que es casi 0.
    filtered_image2 = np.abs(ifft2(fft_image2))

    # Combina las dos imágenes filtradas
    hybrid_image = alpha * filtered_image1 + (1 - alpha) * filtered_image2


    return hybrid_image


In [ ]:
apple1 = cv2.imread('/content/apple1.jpg')
orange1 = cv2.imread('/content/orange1.jpg')

alpha = 0.750
cutoff_frequency = 600

orange_apple=imagen_hibrida_fourier(apple1, orange1, cutoff_frequency, alpha)
cv2.imwrite('/content/orange_apple_Fourier.jpg',orange_apple)

In [ ]:
girl = cv2.imread('girl.jpg')
woman = cv2.imread('woman.jpg')

alpha = 0.5
cutoff_frequency=158

girl_woman=imagen_hibrida_fourier(woman, girl, cutoff_frequency, alpha)
cv2.imwrite('girl_woman_Fourier.jpg',girl_woman)

In [ ]:
obama = cv2.imread('obama.jpg')
michelle = cv2.imread('michelle.jpg')

alpha = 0.35
cutoff_frequency=286

obama_michelle=imagen_hibrida_fourier(michelle, obama, cutoff_frequency, alpha)
cv2.imwrite('/content/obama_michelle_Fourier.jpg',obama_michelle)

In [ ]:
taza = cv2.imread('/content/taza.jpeg')
va = cv2.imread('/content/va.jpeg')

alpha = 0.48
cutoff_frequency=110

taza_va=imagen_hibrida_fourier(va, taza, cutoff_frequency, alpha)
cv2.imwrite('/content/taza_va.jpg',taza_va)